In [48]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

db = DocBin()

In [ ]:
!python -m spacy download en_core_web_lg

In [10]:
nlp=spacy.load('en_core_web_lg')
from spacy.lang.en import English

In [49]:
file = open('/Users/sofiahalima/PycharmProjects/EntityExtractionDemo/data/annotations.json')
training_data = json.load(file)

In [51]:
for text,annotation in tqdm(training_data["annotations"]):
    doc = nlp.make_doc(text)
    entities=[]
    for start, end, label in annotation["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is not None:
            entities.append(span)
            
    doc.ents = entities
    db.add(doc)
    
    db.to_disk("training_data_doc.spacy")

100%|██████████| 19/19 [00:00<00:00, 580.39it/s]


In [ ]:
!python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

In [57]:
!python -m spacy train config.cfg --output output --paths.train training_data_doc.spacy --paths.dev training_data_doc.spacy

/Users/sofiahalima/PycharmProjects/EntityExtractionDemo/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     53.27    9.79    5.60   38.55    0.10
 15     200        281.86   1667.71  100.00  100.00  100.00    1.00
 34     400          6.60      5.78  100.00  100.00  100.00    1.00
 58     600          0.00      0.00  100.00  100.00  100.00    1.0

In [59]:
nlp_news = spacy.load('output/model-best')

In [60]:
doc = nlp_news(''' Vladimir Putin has led Russia's Victory Day commemorations with a parade in Red Square and heightened security after days of Ukrainian strikes targeting the capital.

China's Xi Jinping joined Putin as the Russian president told thousands of soldiers and more than 20 international leaders that Russia remembered the lessons of World War Two.

Putin used his speech to tie the war to today's full-scale invasion of Ukraine, and said all of Russia was behind what he called the "special military operation" - now well into its fourth year.

For the first time, a column of trucks carrying various combat drones took part in the Victory Day parade, apparently because of their widescale use in Ukraine.''')

In [61]:
spacy.displacy.render(doc, style="ent")